In [8]:
!pip install pyspark

In [9]:
import pandas as pd
import random
import os
import sys
from pyspark.context import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
import pyspark.sql.types as pst
from pyspark.ml.feature import Bucketizer
import copy
import pandas as pd
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**促活场景，可以不发优惠券**

In [10]:
user_id = [3790260]*4 + [4687825]*4 + [5077519]*4 + [7927329]*4
coupon_id = [2060114740, 1806204031, 4096652575, 1234832524] * 4
coupon_reduce = [20, 10, 10, 0] * 4
coupon_threshold = [60, 60, 40, -1] * 4
is_convert = []
control_convert = []

In [11]:
for _ in range(4):
  for _ in range(3):
    is_convert.append(random.uniform(0.005, 0.3))
  temp = random.uniform(0, 0.005)
  is_convert.append(temp)
  control_convert.extend([temp]*4)

In [12]:
data = {'user_id': user_id, 'coupon_id': coupon_id, 'coupon_reduce': coupon_reduce, 'coupon_threshold': coupon_threshold, 'is_convert': is_convert, 'control_convert':control_convert}

In [13]:
df_old = pd.DataFrame(data)
df_old['expected_threshold_uplift'] = (df_old['is_convert'] - df_old['control_convert']) * df_old['coupon_threshold']

In [ ]:
df_old

,user_id,coupon_id,coupon_reduce,coupon_threshold,is_convert,control_convert,expeted_uplift
0,3790260,2060114740,20,60,0.123163,0.004987,7.090579
1,3790260,1806204031,10,60,0.031058,0.004987,1.564253
2,3790260,4096652575,10,40,0.046526,0.004987,1.661554
3,3790260,1234832524,0,-1,0.004987,0.004987,-0.000000
4,4687825,2060114740,20,60,0.238143,0.000583,14.253599
5,4687825,1806204031,10,60,0.097641,0.000583,5.823499
6,4687825,4096652575,10,40,0.289030,0.000583,11.537874
7,4687825,1234832524,0,-1,0.000583,0.000583,-0.000000
8,5077519,2060114740,20,60,0.125512,0.001945,7.414024
9,5077519,1806204031,10,60,0.040139,0.001945,2.291634


**拉新场景，都需要发优惠券**

In [14]:
user_id = [3790260]*4 + [4687825]*4 + [5077519]*4 + [7927329]*4
coupon_id = [2060114740, 1806204031, 4096652575, 1234832524] * 4
coupon_reduce = [20, 10, 10, 0] * 4
coupon_threshold = [98, 98, 68, -1] * 4
is_convert = []
control_convert = []

In [15]:
for _ in range(4):
  for _ in range(3):
    is_convert.append(random.uniform(0.005, 0.3))
  temp = 0
  is_convert.append(temp)
  control_convert.extend([temp]*4)

In [16]:
data = {'user_id': user_id, 'coupon_id': coupon_id, 'coupon_reduce': coupon_reduce, 'coupon_threshold': coupon_threshold, 'is_convert': is_convert, 'control_convert':control_convert}

In [17]:
df_new = pd.DataFrame(data)
df_new['expected_threshold_uplift'] = (df_new['is_convert'] - df_new['control_convert']) * df_new['coupon_threshold']

In [18]:
df_new

,user_id,coupon_id,coupon_reduce,coupon_threshold,is_convert,control_convert,expected_threshold_uplift
0,3790260,2060114740,20,98,0.104190,0,10.210650
1,3790260,1806204031,10,98,0.226198,0,22.167426
2,3790260,4096652575,10,68,0.060770,0,4.132368
3,3790260,1234832524,0,-1,0.000000,0,-0.000000
4,4687825,2060114740,20,98,0.275893,0,27.037545
5,4687825,1806204031,10,98,0.268739,0,26.336403
6,4687825,4096652575,10,68,0.127741,0,8.686375
7,4687825,1234832524,0,-1,0.000000,0,-0.000000
8,5077519,2060114740,20,98,0.239685,0,23.489112
9,5077519,1806204031,10,98,0.096759,0,9.482382


**转为pyspark计算**

In [19]:
# 初始化spark会话
spark = SparkSession.builder.getOrCreate()

In [20]:
spark_df_old = spark.createDataFrame(df_old)
spark_df_old.show()

+-------+----------+-------------+----------------+--------------------+--------------------+-------------------------+
|user_id| coupon_id|coupon_reduce|coupon_threshold|          is_convert|     control_convert|expected_threshold_uplift|
+-------+----------+-------------+----------------+--------------------+--------------------+-------------------------+
|3790260|2060114740|           20|              60| 0.12152982271486865|0.003223793913013...|        7.098361728111288|
|3790260|1806204031|           10|              60| 0.17571044189891105|0.003223793913013...|       10.349198879153834|
|3790260|4096652575|           10|              40| 0.23093566493430362|0.003223793913013...|        9.108474840851592|
|3790260|1234832524|            0|              -1|0.003223793913013...|0.003223793913013...|                     -0.0|
|4687825|2060114740|           20|              60| 0.10331382449754938|0.001300859360866...|        6.120777908200959|
|4687825|1806204031|           10|      

In [21]:
spark_df_new = spark.createDataFrame(df_new)
spark_df_new.show()

+-------+----------+-------------+----------------+--------------------+---------------+-------------------------+
|user_id| coupon_id|coupon_reduce|coupon_threshold|          is_convert|control_convert|expected_threshold_uplift|
+-------+----------+-------------+----------------+--------------------+---------------+-------------------------+
|3790260|2060114740|           20|              98| 0.10419030549147965|              0|       10.210649938165005|
|3790260|1806204031|           10|              98| 0.22619822363655365|              0|       22.167425916382257|
|3790260|4096652575|           10|              68|0.060770111562467394|              0|        4.132367586247783|
|3790260|1234832524|            0|              -1|                 0.0|              0|                     -0.0|
|4687825|2060114740|           20|              98|  0.2758933186869389|              0|        27.03754523132001|
|4687825|1806204031|           10|              98| 0.26873880920353477|        

In [22]:
result_df_new = spark_df_new.groupby('user_id').agg(
    F.collect_list('is_convert').alias('is_convert'),
    F.collect_list('coupon_id').alias('coupon_id'),
    F.collect_list('coupon_reduce').alias('coupon_reduce'),
    F.collect_list('coupon_threshold').alias('coupon_threshold'),
    F.collect_list('expected_threshold_uplift').alias('expected_threshold_uplift'),).cache()

result_df_new.show()

+-------+--------------------+--------------------+---------------+----------------+-------------------------+
|user_id|          is_convert|           coupon_id|  coupon_reduce|coupon_threshold|expected_threshold_uplift|
+-------+--------------------+--------------------+---------------+----------------+-------------------------+
|3790260|[0.10419030549147...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|     [10.2106499381650...|
|4687825|[0.27589331868693...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|     [27.0375452313200...|
|5077519|[0.23968481145875...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|     [23.4891115229580...|
|7927329|[0.03140481379917...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|     [3.07767175231930...|
+-------+--------------------+--------------------+---------------+----------------+-------------------------+



In [23]:
result_df_old = spark_df_old.groupby('user_id').agg(
    F.collect_list('is_convert').alias('is_convert'),
    F.collect_list('coupon_id').alias('coupon_id'),
    F.collect_list('coupon_reduce').alias('coupon_reduce'),
    F.collect_list('coupon_threshold').alias('coupon_threshold'),
    F.collect_list('expected_threshold_uplift').alias('expected_threshold_uplift'),).cache()

result_df_old.show()

+-------+--------------------+--------------------+---------------+----------------+-------------------------+
|user_id|          is_convert|           coupon_id|  coupon_reduce|coupon_threshold|expected_threshold_uplift|
+-------+--------------------+--------------------+---------------+----------------+-------------------------+
|3790260|[0.12152982271486...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|     [7.09836172811128...|
|4687825|[0.10331382449754...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|     [6.12077790820095...|
|5077519|[0.15068910340136...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|     [8.88552251144493...|
|7927329|[0.28159564187324...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|     [16.6940647108250...|
+-------+--------------------+--------------------+---------------+----------------+-------------------------+



**MCKP求解**

In [24]:
epoch = 1000
B = 50
C = 3 * B
alpha = 1 / B
lr_decay = 0.98

In [25]:
def MCKP(df, epoch, B, C, alpha, lr_decay=1):
    N = df.count()
    lambda_B = 0
    lambda_C = 0

    for _ in tqdm(range(epoch)):

        temp = df.cache()

        # 固定lambda，更新xij，选择收益最大的优惠券
        temp = temp.withColumn('score', F.expr('zip_with(is_convert, coupon_reduce, (x, y) -> (x - %f * y))' % (lambda_B)))
        temp = temp.withColumn('score', F.expr('zip_with(score, expected_threshold_uplift, (x, y) -> (x + %f * y))' % (lambda_C)))
        temp = temp.withColumn('choice', F.expr('array_position(score, array_max(score)) - 1'))

        # 固定优惠券，更新lambda
        delta_B = temp.select(F.sum(F.col('coupon_reduce')[F.col('choice')])).collect()[0][0]
        lambda_B = max(0, lambda_B - alpha * (B - delta_B)/N)

        delta_C = temp.select(F.sum(F.col('expected_threshold_uplift')[F.col('choice')])).collect()[0][0]
        lambda_C = max(0, lambda_C - alpha * (delta_C - C)/N)

        if _ % 100 == 0:
            print("epoch: {0}, lambda_B: {1}, lambda_C: {2}".format(_, lambda_B, lambda_C))

        alpha *= lr_decay
        temp.unpersist()

    return temp

In [26]:
result_old = MCKP(result_df_old, epoch, B, C, alpha, lr_decay)

  0%|          | 1/1000 [00:01<26:04,  1.57s/it]

epoch: 0, lambda_B: 0, lambda_C: 0.526190356709408


 10%|█         | 101/1000 [00:38<03:44,  4.01it/s]

epoch: 100, lambda_B: 0, lambda_C: 22.62648035274624


 20%|██        | 201/1000 [01:07<02:54,  4.59it/s]

...

 80%|████████  | 802/1000 [03:16<00:28,  7.02it/s]

epoch: 800, lambda_B: 0, lambda_C: 26.005538008001007


 90%|█████████ | 902/1000 [03:32<00:13,  7.07it/s]

epoch: 900, lambda_B: 0, lambda_C: 26.005540122748528


100%|██████████| 1000/1000 [03:48<00:00,  4.38it/s]


In [27]:
result_old = result_old.withColumn('choose_coupon', F.col('coupon_id')[F.col('choice')])

In [28]:
result_old.show()

+-------+--------------------+--------------------+---------------+----------------+-------------------------+--------------------+------+-------------+
|user_id|          is_convert|           coupon_id|  coupon_reduce|coupon_threshold|expected_threshold_uplift|               score|choice|choose_coupon|
+-------+--------------------+--------------------+---------------+----------------+-------------------------+--------------------+------+-------------+
|3790260|[0.12152982271486...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|     [7.09836172811128...|[184.718259677582...|     1|   1806204031|
|4687825|[0.10331382449754...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|     [6.12077790820095...|[159.277448547333...|     2|   4096652575|
|5077519|[0.15068910340136...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|     [8.88552251144493...|[231.223500195683...|     2|   4096652575|
|7927329|[0.28159564187324...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1

In [29]:
result_new = MCKP(result_df_new, epoch, B, C, alpha, lr_decay)

  0%|          | 1/1000 [00:00<06:49,  2.44it/s]

epoch: 0, lambda_B: 0.05, lambda_C: 0.25709545049409543


 10%|█         | 102/1000 [00:18<02:21,  6.36it/s]

epoch: 100, lambda_B: 0.795185903112614, lambda_C: 11.280824493162205


 20%|██        | 202/1000 [00:34<01:47,  7.42it/s]

...

 80%|████████  | 801/1000 [02:07<00:58,  3.43it/s]

epoch: 800, lambda_B: 0.9098345092364453, lambda_C: 12.96626447283403


 90%|█████████ | 901/1000 [02:21<00:17,  5.60it/s]

epoch: 900, lambda_B: 0.9098345092364453, lambda_C: 12.96626553267963


100%|██████████| 1000/1000 [02:36<00:00,  6.40it/s]


In [30]:
result_new = result_new.withColumn('choose_coupon', F.col('coupon_id')[F.col('choice')])

In [31]:
result_new.show()

+-------+--------------------+--------------------+---------------+----------------+-------------------------+--------------------+------+-------------+
|user_id|          is_convert|           coupon_id|  coupon_reduce|coupon_threshold|expected_threshold_uplift|               score|choice|choose_coupon|
+-------+--------------------+--------------------+---------------+----------------+-------------------------+--------------------+------+-------------+
|3790260|[0.10419030549147...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|     [10.2106499381650...|[114.301493436622...|     1|   1806204031|
|4687825|[0.27589331868693...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|     [27.0375452313200...|[332.655196775013...|     1|   1806204031|
|5077519|[0.23968481145875...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|     [23.4891115229580...|[286.609052921797...|     0|   2060114740|
|7927329|[0.03140481379917...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1